# 線形化ベクトル方程式への線形化

吉田勝俊（宇都宮大学）

### 参考情報
- [Welcome to SymPy’s documentation! &#8212; SymPy 1.10.1 documentation](https://docs.sympy.org/) （本家）
- [SymPy による数式処理とグラフ作成 - 弘前大学 Home Sweet Home](https://home.hirosaki-u.ac.jp/jupyter/sympy/)

In [ ]:
import sympy as sym #数式処理ライブラリ

## ◯オイラー・ラグランジュ方程式

### 定義：

$\displaystyle
\frac{d}{dt}\left(\frac{\partial L}{\partial \dot q_i}\right)
-\frac{\partial L}{\partial q_i}
+\frac{\partial D}{\partial \dot q_i}
-Q_i
= 0, \quad i=1,2,\cdots
$

In [ ]:
def derive_ELE(system):
    '''
    オイラー・ラグランジュ方程式（ELE: Euler-Lagrange equation）
    「左辺＝０」の左辺を導出する関数
    
    def system():    
        q: 一般化座標（ベクトル）
        L: ラグランジュ関数（スカラ）
        D: 散逸関数（スカラ）
        Q: 一般化力（ベクトル）
        u: 外部入力（ベクトル）※線形化で用いる
        
        return (t, q, L, D, Q, u)
    '''
    t, q, L, D, Q, u = system()       #一般化座標, ラグランジアン, 散逸関数, 一般化力
    dqdt             = sym.diff(q, t) #一般化座標の時間微分
    dim              = len(q)         #一般化座標の次元
    
    ### ラグランジュの運動方程式 ###
    ELE = sym.Matrix([
        sym.simplify(
            sym.diff( sym.diff(L, dqdt[i]), t )
            - sym.diff(L, q[i])
            + sym.diff(D, dqdt[i])
            - (Q[i] if Q is not None else 0) #Q が無ければ 0
        )
        for i in range(dim)
    ])

    return (ELE, q, t, u)

## ◯ベクトル運動方程式への線形化

In [ ]:
def initial_state(x0, x0symb, x):
    '''
    線形化の基準点が未指定なら記号で生成する
    '''
    if x0 is None:
        if len(x) == 1:  #基準点がスカラ（1次元）の場合
            x0 = sym.Matrix([x0symb])
        else:            #基準点がベクトル（多次元）の場合
            x0 = sym.Matrix([
                sym.symbols(x0symb + '_' + str(i+1)) for i in range(len(x))
            ])

    else:
        x0 = sym.Matrix(x0)

    return x0

In [ ]:
def LinearizedEOM(ELE, q0=None, u0=None):
    '''
    運動方程式を，x0 の近傍で線形化する
    - ELE := derive_ELE(system)の出力
    '''
    EOMq, q, t, u = ELE

    #
    # 一般化座標の微分
    #
    dq  = sym.Matrix([sym.diff(qi, t) for qi in q])    #1階微分
    ddq = sym.Matrix([sym.diff(qi, t, 2) for qi in q]) #2階微分
    
    #
    # ヤコビ行列
    #
    
    M = EOMq.jacobian(ddq) #慣性行列：加速度に関するヤコビ行列
    C = EOMq.jacobian(dq)  #減衰行列：速度に関するヤコビ行列
    K = EOMq.jacobian(q)   #剛性行列：変位に関するヤコビ行列
    
    #入力行列 ※これだけ右辺なのでマイナス　
    U = -EOMq.jacobian(u) if u is not None else None

    # 平衡点の準備
    q0 = initial_state(q0, 'q0', q)
    u0 = initial_state(u0, 'u0', u) if u is not None else None

    # 平衡点と平衡条件の代入
    def subs_equilibrium(mat):
        '''
        平衡点と平衡条件を代入する
        '''
        for i in range(len(q)):
            mat = mat.subs(q[i], q0[i])
            mat = mat.subs(dq[i], 0)
            mat = mat.subs(ddq[i], 0)
        
        if u is not None:
            for i in range(len(u)):
                mat = mat.subs(u[i], u0[i]) 
            
        return sym.simplify(mat)
    
    M = subs_equilibrium(M)
    C = subs_equilibrium(C)
    K = subs_equilibrium(K)
    if u is not None:
        U = subs_equilibrium(U)
    
    return (M, C, K, U)

### ■単振り子（SP: simple pendulum）

- 一般化座標: $(\theta(t))$

In [ ]:
def system_SP():
    '''
    システムを定義する関数（単振り子）
    '''
    # パラメータ: 
    t, m, l, g = sym.symbols(
        't m l g', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    c = sym.symbols('c', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    theta = q[0]
    xx = l*sym.Matrix([
        sym.sin(theta),
        -sym.cos(theta),
    ])
    
    dxxdt = sym.diff(xx, t) #その時間微分
    
    # 運動エネルギー
    T = (m/2)*dxxdt.dot(dxxdt)
    
    # 位置エネルギー
    h = xx[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
#     D = (c/2)*dxxdt.dot(dxxdt) #粘性減衰
    D = 0 #減衰なし

    # 外部入力（この系は無し）
    u = None

    # 一般化力
    Q = u
    
    return (t, q, L, D, Q, u)

#### 運動方程式の導出結果

In [ ]:
ELE_SP = derive_ELE(system_SP)

for _ in ELE_SP: display(_)

#### 線形化の結果

In [ ]:
linearized_SP = LinearizedEOM(ELE_SP)
for _ in linearized_SP: display(_)

出力にある行列により，$\boldsymbol{x}_0 = (x_{01},x_{02})^T$の近傍で線形化した状態方程式は次のように表せます．

$$
\begin{bmatrix}
 \dot \xi_1 \\ \dot \xi_2
\end{bmatrix}
=
\begin{bmatrix}
 0 & 1 \\
 -\frac{g\cos x_{01}}{l} & 0 \\
\end{bmatrix}
\begin{bmatrix}
 \xi_1 \\ \xi_2
\end{bmatrix}
$$

### ■台車型倒立振子（CIP: cart inverted pendulum）

- 一般化座標: $(x(t),\theta(t))$

In [ ]:
def system_CIP():
    '''
    システムを定義する関数 （問題に応じて書き換える）
    ※以下は，台車型倒立振子の例
    '''
    # パラメータ: 
    t, M, m, l, g, c1, c2 = sym.symbols(
        't M m l g c_1 c_2', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    G, S = sym.symbols('G S', positive=True)
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'x', 
            real=True #実数値に制限
        )(t),
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    x, th = q
    xM = sym.Matrix([
        x,
        G
    ])
    xm = sym.Matrix([
        x + l*sym.sin(th),
        l*sym.cos(th) + S,
    ])
    
    dxMdt = sym.diff(xM, t) #その時間微分
    dxmdt = sym.diff(xm, t)
    
    # 運動エネルギー
    T = (M/2)*dxMdt.dot(dxMdt) + (m/2)*dxmdt.dot(dxmdt)
    
    # 位置エネルギー
    h = xm[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    D = (c1/2)*sym.diff(x,t)**2 + (c2/2)*sym.diff(th,t)**2 #粘性減衰
#     D = 0 #減衰なし

    # 外部入力（この系は１個だけ）
    u = sym.Matrix([
        sym.Function(r'u', real=True)(t)
    ])
     
    # 一般化力（外部入力が第１成分のみに作用）
    Q = sym.Matrix([
        u[0],
        0
    ])
    
    return (t, q, L, D, Q, u)

#### 運動方程式の導出結果

In [ ]:
ELE_CIP = derive_ELE(system_CIP)

for _ in ELE_CIP: display(_)

#### 線形化の結果

In [ ]:
linearized_CIP = LinearizedEOM(ELE_CIP) #平衡点未指定
for _ in linearized_CIP: display(_) 

In [ ]:
linearized_CIP = LinearizedEOM(ELE_CIP, [0,0], [0])
for _ in linearized_CIP: display(_) 